In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

In [2]:
class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model, dropout, maxlen=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(maxlen, d_model)
        position = torch.arange(0, maxlen, dtype=torch.float).unsqueeze(1)
        div_model = torch.exp(torch.arange(0, maxlen, 2).float() *
                              (-math.log(10000)/d_model))
        pe[:, 0::2] = torch.sin(position * div_model)
        pe[:, 1::2] = torch.cos(position * div_model)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [3]:
class TransformerModel(nn.Module):
    
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.encoder = nn.Embedding(ntoken, ninp)
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid,
                                                 dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers,
                                                      nlayers)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)
        
        self.init_weights()
        
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
        
    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(
            mask==0, float('-inf')).masked_fill(mask==1, float(0.0))
        return mask
    
    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [6]:
import torchtext
from torchtext.data.utils import get_tokenizer

TEXT = torchtext.data.Field(tokenize=get_tokenizer('spacy'),
                            init_token='<sos>', eos_token='<eos>',
                            lower=True)

In [7]:
type(TEXT)

torchtext.data.field.Field

In [8]:
%time train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)

downloading wikitext-2-v1.zip
extracting
CPU times: user 13.5 s, sys: 455 ms, total: 13.9 s
Wall time: 31.8 s


In [9]:
type(train_txt)

torchtext.datasets.language_modeling.WikiText2

In [10]:
TEXT.build_vocab(train_txt)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [20]:
len(train_txt.examples[0].text)

2236652

In [23]:
TEXT.numericalize([train_txt.examples[0].text], device=device)

RuntimeError: Invalid device, must be cuda device